In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import csv
from io import StringIO
import json



In [ ]:
#url to reqest the information
url='https://finance.yahoo.com/quote/{}?p={}&.tsrc=fin-srch'

In [ ]:
#make request for the given webpage
tick='EDU'
response=requests.get(url.format(tick,tick))
soup=BeautifulSoup(response.text,'html.parser')         #creating the soup object with given page
print(soup)
#print(response)

categories=['most-active','gainers','losers'] #based on yahoo page
#categories[0]

In [ ]:
response=requests.get(url.format(categories[1]))        #request for a web page
soup=BeautifulSoup(response.text,'html.parser')         #creating the soup object with given page
print(soup)

In [ ]:
soup=BeautifulSoup(response.text,'html.parser')         #creating the soup object with given page

In [ ]:
#soup.prettify() #creates the nice print
soup.title.string

In [ ]:
#find the ticker block
table=soup.find('table',class_="W(100%)")
table_body=table.find('tbody')
table_data_row=table_body.find('tr')
table_data=table_data_row.find_all('td') #find all returns the list

#link="https://finance.yahoo.com"+href
#print(link)
tick_info=[]
for data in table_data[:-1]:
    #print("data\t",data)
    
    #print("data text\t",data.text)
    
    tick_info.append(data.text)
    #tikc_info.append(link)
href=table_data[0].find('a').get('href')
#href=data.find('a').get('href')
link="https://finance.yahoo.com"+href
tick_info.append(link)
print("link\t",link)
tick_info

In [ ]:
%%script false --no-raise-error #skip this cell and no error output
#csv file to save the data
csv_file=open("actives.csv","w")
header=['Symbol','Name','Price','Change','%Change','Volume','Avg Vol(3 mo)','Market Cap','PE (TTM)'] #columns name
csv_writer=csv.writer(csv_file)
csv_writer.writerow(header)
#find the ticker block
table=soup.find('table',class_="W(100%)")
table_body=table.find('tbody')
table_data_rows=table_body.find_all('tr')    #get all the rows
all_data=[]
for row in table_data_rows:
    table_data=row.find_all('td')[:-1] #find all returns the list
    tick_info=[]
    for data in table_data:
        tick_info.append(data.text)
    all_data.append(tick_info)
#tick_info
#all_data
#len(all_data)
#print(all_data[0])
for i in range(len(all_data)):
    print(all_data[i])
    csv_writer.writerow(all_data[i])
csv_file.close()

In [ ]:
%%script false --no-raise-error #skip this cell and no error output
#most active data
#csv file to save the data
csv_file=open("actives.csv","w")
header=['Symbol','Name','Price','Change','%Change','Volume','Avg Vol(3 mo)','Market Cap','PE (TTM)','Link'] #columns name
csv_writer=csv.writer(csv_file)
csv_writer.writerow(header)
#find the ticker block
table=soup.find('table',class_="W(100%)")
table_body=table.find('tbody')
table_data_rows=table_body.find_all('tr')    #get all the rows
all_data=[]
for row in table_data_rows:
    table_data=row.find_all('td')[:-1] #find all returns the list
    tick_info=[]
    for data in table_data:
        tick_info.append(data.text)
    href=table_data[0].find('a').get('href')
    link="https://finance.yahoo.com"+href
    tick_info.append(link)
    all_data.append(tick_info)
#tick_info
#all_data
#len(all_data)
#print(all_data[0])
for i in range(len(all_data)):
    print(all_data[i])
    csv_writer.writerow(all_data[i])
csv_file.close()

In [ ]:
#csv file to save the data
csv_file=open("temp/gainers.csv","w")
header=['Symbol','Name','Price','Change','%Change','Volume','Avg Vol(3 mo)','Market Cap','PE (TTM)','Link'] #columns name
csv_writer=csv.writer(csv_file)
csv_writer.writerow(header)
#find the ticker block
table=soup.find('table',class_="W(100%)")
table_body=table.find('tbody')
table_data_rows=table_body.find_all('tr')    #get all the rows
all_data=[]
for row in table_data_rows:
    table_data=row.find_all('td')[:-1] #find all returns the list
    tick_info=[]
    for data in table_data:
        tick_info.append(data.text)
    href=table_data[0].find('a').get('href')
    link="https://finance.yahoo.com"+href
    tick_info.append(link)
    all_data.append(tick_info)
#tick_info
#all_data
#len(all_data)
#print(all_data[0])
for i in range(len(all_data)):
    print(all_data[i])
    csv_writer.writerow(all_data[i])
csv_file.close()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#make clickable link for the column

def make_clickable(val):
    return '<a target ="_blank" href="{}">{}</a>'.format(val,"link")
    #target="_blank" opens the window to new window
    #can use val inplace to "link" to print the exact link



In [ ]:
def color_negative_red(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    color = 'red' if val < 0 else 'green'
    return 'color: %s' % color

In [ ]:
def change_volume(v):
    '''
    Changes the volume into million if not already so.
    '''
    if v[-1] not in ['M','B']:
        if ',' in v:
            idx=v.find(',') 
            v=v[:idx]+v[idx+1:]
        sub=v[:-1]
        sub=float(sub)/10**6
        sub=str(sub)+'M'
    else:sub=v
    return sub

In [ ]:
test='100,01'
change_volume(test)

In [ ]:
#df_actives=pd.read_csv('actives.csv')
df_losers=pd.read_csv('temp/gainers.csv')
df_losers.head()
#df_losers.style.format({'Link':make_clickable},{'Change':color_negative_red})#column names and formatting as the dictonary
#df_losers
df_losers['Volume']=df_losers['Volume'].apply(change_volume)
df_losers['Avg Vol(3 mo)']=df_losers['Avg Vol(3 mo)'].apply(change_volume)
df_losers=df_losers.style.\
applymap(color_negative_red,subset=['Change']).\
format({'Link':make_clickable}).\
format({'Price':"{:.2f}"}).\
format({'Change':"{:.2f}"})



In [ ]:
df_losers.columns

In [ ]:
df_losers

In [ ]:
#data types of the data-frame
#df_losers.dtypes
#df_losers.style.applymap(color_negative_red,subset=['Change'])
#builtin method to give color to the null values
#https://mode.com/example-gallery/python_dataframe_styling/ example
#df_losers.style.highlight_null(null_color='re')

In [ ]:
df_gainers=df_gainers.style.format({'Link':make_clickable})#column names and formatting as the dictonary
df_gainers

In [ ]:
#df_actives=df_actives.style.format({'Link':make_clickable})#column names and formatting as the dictonary
#df_actives=df_actives.style.format({'Change':color_tick_red})
#df_actives

In [ ]:
df_actives.format("{:.2f}")

In [ ]:
#%%script false --no-raise-error #skip this cell and no error output
#ticker='NIO'
def color_tick_red(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for given value
    strings, black otherwise.
    """
    color = 'red' if val <0 else 'black'
    return 'color: %s' % color

#s = df_most_active.style.applymap(color_tick_red)
#s

In [ ]:
df_actives.style#.apply(color_tick_red,subset=['Change'])

# Use Line wrap to see all the lines
# /* -- Data -- */ includes the data

In [ ]:
#getting the pattern with regular expression
pattern=re.compile(r'\s--\sData\s--\s') #\s->space
script_data=soup.find('script',text=pattern).contents[0]

In [ ]:
script_data[:500]

In [ ]:
script_data[-500:]

In [ ]:
#Extracting the json data from the text 
start=script_data.find("context")-2 # first { positon
json_data=json.loads(script_data[start:-12]) # {  is last 12 characters from the back
#json loads gives python dictionary

In [ ]:
json_data['context'].keys()

In [ ]:
link='https://www.timesjobs.com/candidate/job-search.html?searchType=personalizedSearch&from=submit&txtKeywords=python&txtLocation='
html_text=requests.get(link).text
soup=BeautifulSoup(html_text,'lxml')
jobs=soup.find_all('li',class_='clearfix job-bx wht-shd-bx') #find first occurance


In [ ]:
for job in jobs:
    date_published=job.find('span',class_="sim-posted").span.text
    #print(date_published)
    if 'few' in date_published:
        company_name=job.find('h3',class_="joblist-comp-name").text.replace(" ","")
        #print(jobs)<span class="srp-skills">
        #skills=job.find('span', class_="srp-skills")
        #print(skills)
        #print(company_name)
        skills=job.find('span', class_="srp-skills").text.replace(" ","")
        more_info=job.header.h2.a['href']


        #print(skills)
        #print(company_name)
        print(f'''
        Company Name: {company_name.strip()}
        Required Skills:{skills.strip()}
        More_Information:{more_info}
          '''
        )
        print()

In [ ]:
print(html_text)